# Fine-tune Llama 2


In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/Users/tejaswinithota/Desktop/Question-Generation-With-LLMs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tejaswinithota/Desktop/Question-Generation-With-LLMs/myenv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "lmqg/qg_squad"

# Fine-tuned model name
new_model = "llama-2-7b-qa-gen"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [3]:
def format_data(sample):
    question = f"{sample['question']}"
    context = f"<s>[INST]{sample['paragraph']}[/INST]" if len(sample["paragraph"]) > 0 else None
    response = f"{sample['sentence_answer']}"
    # join all the parts together
    prompt = "".join([i for i in [context, question, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_data(sample)}{tokenizer.eos_token}"
    return sample


In [6]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16 and CUDA availability
if torch.cuda.is_available():
    if compute_dtype == torch.float16 and use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16: accelerate training with bf16=True")
            print("=" * 80)
else:
    print("CUDA is not available. Training will proceed on CPU.")

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

dataset = load_dataset("lmqg/qg_squad", split="train")

# Shuffle the dataset
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
dataset = dataset_shuffled.select(range(50))

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=50,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)


CUDA is not available. Training will proceed on CPU.


KeyboardInterrupt: 

In [15]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [16]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
# prompt = "What is a large language model?"
prompt = "Amidst the vast expanse of the cosmos, a lone spacecraft drifts through the void, its hull battered and its systems failing. Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation. As they chart their course through the unfathomable darkness, they are haunted by memories of Earth and the loved ones they left behind. But amidst the despair, a glimmer of hope remains. Generate Question and Answers from given information."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=600)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'].split('<hl>'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] Amidst the vast expanse of the cosmos, a lone spacecraft drifts through the void, its hull battered and its systems failing. Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation. As they chart their course through the unfathomable darkness, they are haunted by memories of Earth and the loved ones they left behind. But amidst the despair, a glimmer of hope remains. Generate Question and Answers from given information. [/INST]  Sure, I'd be happy to help generate questions and answers based on the information provided about the spacecraft and its crew:

Question: What is happening inside the spacecraft? <hl> Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation. <hl> </s> "Among the vast expanse of the cosmos, a lone spacecraft drifts through the void, its hull battered and its systems failing. Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation." As they chart t

In [21]:
#Sample Test
print(result[0]['generated_text'].split('<hl>')[0])
print("Answer: ",result[0]['generated_text'].split('<hl>')[1])

<s>[INST] Amidst the vast expanse of the cosmos, a lone spacecraft drifts through the void, its hull battered and its systems failing. Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation. As they chart their course through the unfathomable darkness, they are haunted by memories of Earth and the loved ones they left behind. But amidst the despair, a glimmer of hope remains. Generate Question and Answers from given information. [/INST]  Sure, I'd be happy to help generate questions and answers based on the information provided about the spacecraft and its crew:

Question: What is happening inside the spacecraft? 
Answer:   Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation. 


In [17]:
print(result)

[{'generated_text': '<s>[INST] Amidst the vast expanse of the cosmos, a lone spacecraft drifts through the void, its hull battered and its systems failing. Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation. As they chart their course through the unfathomable darkness, they are haunted by memories of Earth and the loved ones they left behind. But amidst the despair, a glimmer of hope remains. Generate Question and Answers from given information. [/INST]  Sure, I\'d be happy to help generate questions and answers based on the information provided about the spacecraft and its crew:\n\nQuestion: What is happening inside the spacecraft? <hl> Inside, the crew grapples with dwindling supplies and the ever-looming specter of isolation. <hl> </s> "Among the vast expanse of the cosmos, a lone spacecraft drifts through the void, its hull battered and its systems failing. Inside, the crew grapples with dwindling supplies and the ever-looming specter of iso

In [18]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

20933

In [19]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]